### Load your data

In [2]:
from langchain.document_loaders import UnstructuredPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

loader = UnstructuredPDFLoader(r"C:\Users\s39721\repos\ChatPTK\data\avtalat_handbok_kollektivavtalad-tjanstepension-och-forsakring_010123.pdf")
data = loader.load()

detectron2 is not installed. Cannot use the hi_res partitioning strategy. Falling back to partitioning with the fast strategy.


In [3]:
print (f'You have {len(data)} document(s) in your data')
print (f'There are {len(data[0].page_content)} characters in your document')

You have 1 document(s) in your data
There are 276579 characters in your document


### Chunk your data up into smaller documents

In [4]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(data)

In [5]:
print (f'Now you have {len(texts)} documents')

Now you have 324 documents


### Create embeddings of your documents to get ready for semantic search

In [6]:
from langchain.vectorstores import Pinecone
from langchain.embeddings.openai import OpenAIEmbeddings
import pinecone

c:\Users\s39721\repos\ChatPTK\.venv\lib\site-packages\pinecone\index.py:4: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [1]:
import os
import getpass

os.environ['OPENAI_API_KEY'] = getpass.getpass(prompt='OpenAI API Key:')
os.environ['PINECONE_API_KEY'] = getpass.getpass(prompt='Pinecone API Key:')
os.environ['PINECONE_API_ENV'] = 'us-west4-gcp'

In [8]:
embeddings = OpenAIEmbeddings(openai_api_key=os.environ['OPENAI_API_KEY'])

In [9]:
# initialize pinecone
pinecone.init(
    api_key=os.environ['PINECONE_API_KEY'],  # find at app.pinecone.io
    environment=os.environ['PINECONE_API_ENV']  # next to api key in console
)
index_name = "chatptk" # put in the name of your pinecone index here

In [31]:
# docsearch = Pinecone.from_texts([t.page_content for t in texts], embeddings, index_name=index_name)
docsearch = Pinecone.from_existing_index(index_name=index_name, embedding=embeddings)

Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised RateLimitError: You exceeded your current quota, please check your plan and billing details..


In [15]:
query = "När kan jag som tidigast gå i pension?"
docs = docsearch.similarity_search(query, include_metadata=True)

In [16]:
docs[0].page_content[:250]

'UTBETALNING AV KOLLEKTIVAVTALAD TJÄNSTEPENSION Det pensionsbolag som förvaltar pensionen kontaktar den anställda några månader före 65-årsdagen eller före 66-årsdagen för den som omfattas av ITP 1. Hen får tjänstepension i förhållande till inkomst oc'

### Query those docs to get your answer back

In [18]:
from langchain.llms import OpenAI
from langchain.chains.question_answering import load_qa_chain

In [19]:
llm = OpenAI(temperature=0, openai_api_key=os.environ['OPENAI_API_KEY'])
chain = load_qa_chain(llm, chain_type="stuff")

In [20]:
def run_query(query):
    docs = docsearch.similarity_search(query, include_metadata=True)
    return chain.run(input_documents=docs, question=query)

In [22]:
run_query("Hur skiljer sig sjukförsäkring åt mellan ITP1 och ITP2?")

' ITP2 har en premiebefrielseförsäkring som kan ta över arbetsgivarens inbetalningar av premier till ITP2 vid sjukskrivning från månaden efter dag 90 eller om den anställda varit sjuk i perioder om sammanlagt 105 dagar. ITP1 har inte premiebefrielseförsäkring.'

In [23]:
run_query("Vad är premiebefrielseförsäkring?")

' Premiebefrielseförsäkring är en försäkring som gäller vid sjukdom och som tar över inbetalning av premier till tjänstepension som arbetsgivaren skulle betalat in när den anställda varit på arbetet och fått lön. Den gäller från att den anställda fyller 22 år och tills månaden innan den anställda fyller 65 år. Försäkringen är kopplad till tjänstepension som finns i kollektivavtalet och är proportionell till graden av arbetsoförmåga.'

In [24]:
run_query("Hur mycket betalar min arbetsgivare i premie om jag har ITP1?")

' Arbetsgivaren betalar in pensionspremien ITP 1 är helt premiebestämd. Premien är 4,5 % på delar av lönen upp till 7,5 inkomstbasbelopp (46 438 kronor per månad, år 2023). För delar av lön från 7,5 upp till 30 inkomstbasbelopp är premie 30 %.'

In [25]:
run_query("Hur fungerar tiotaggarval?")

' Tiotaggarval är en tjänstepension som ersätter delar av den kollektivavtalade tjänstepensionen ITP 2. För att kunna välja en alternativ ITP måste den anställda tjäna mer än tio inkomstbasbelopp om året. Arbetsgivaren och den anställda måste vara överens, men det är arbetsgivaren som bestämmer om alternativ ITP ska erbjudas. Den anställda måste komma överens med sin arbetsgivare om att ersätta vissa delar av ITP 2, över 7,5 inkomstbasbelopp med en alternativ ITP (så kallad tio taggarlösning). Det går också att helt byta till ITP 1.'